In [14]:
from datetime import timedelta, date
import math
import pandas as pd
import yfinance as yf
from sympy.solvers import solve
from sympy import Symbol

In [15]:
def ticker(symbol):
    t = yf.Ticker(symbol)
    return t

In [16]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [24]:
# create bank account class
class bank_account: 
    
    # start account with zero money
    def __init__(self): 
        self.balance = 0
    
    # function to deposite amount 
    def deposit(self, amount): 
        self.balance = self.balance + amount 
        print('Amount Deposited: {}'.format(amount))

    # function to withdraw the amount 
    def withdraw(self, amount): 
        print("you are withdrawing an amount of {} out of {}".format(amount, self.balance))
        if self.balance >= amount: 
            self.balance = self.balance - amount 
            print('\n You Withdrew: {}'.format(amount)) 
        else: 
            print("\n Insufficient balance.") 

    # function to return value as amount 
    def balance(self):
        balance = self.balance
        return balance
            
    # function to display the amount 
    def display(self): 
        print('\n Net Available Balance: {}'.format(self.balance))
        
    # function resetting the balance to zero
    def reset(self):
        self.balance = 0
        print('\n Balance reset to zero: {}'.format(self.balance))

In [25]:
s = bank_account()
s.deposit(1000)
print(s.balance)

Amount Deposited: 1000
1000


In [26]:
import yfinance as yf

def ticker(symbol):
    """ Return a ticker with relevant information from the yfinance api. """
    t = yf.Ticker(symbol)
    return t

def price(ticker_name, day):
    # how many consecutive days down before we will buy (test for 1-5)
    SCOPE = 3 # 1, 2 werken niet vanwege weekenden en missende dagen

    t = ticker(ticker_name)
    h = t.history(start = day - timedelta(days = SCOPE), end = day)
    closing_price = h['Close'][-1]
    
    return closing_price
    
    
def buy(ticker_name, day):
    """ For a given ticker, for a given day, check that ticker's history and return a buy / no-buy advice. """
    
    # how many consecutive days down before we will buy (test for 1-5)
    SCOPE = 5 # 1, 2 werken niet vanwege weekenden en missende dagen

    t = ticker(ticker_name)
    h = t.history(start = day - timedelta(days = SCOPE), end = day)
    price = h['Close'][-1]
    
    # add column with change in closing price
    h['Difference'] = h['Close'].diff(periods = 1)
    
    if any(h['Difference'] > 0) == False: # if down in last five days 
        buy = True
    else:
        buy = False
    return buy, price

def sell(ticker_name, no_stocks):
    
    if no_stocks <= 0:
        sell = False
    elif no_stocks > 0:
        sell = True
    return sell

In [27]:
def bep(balance, price_buy):
    
    x = Symbol('x') # selling price
    n = Symbol('n') # no_stock
    
    MARKUP = 1.05
    VARIABLE_COST = 0.01
    FIXED_COST    = 7.9
    
    #######################
    # fixed cost analysis #
    #######################
    cost_buy = FIXED_COST
    cost_sell = FIXED_COST
    total_cost = cost_buy + cost_sell
    no_stocks = (balance - total_cost) / price_buy

    x = Symbol('x') # selling price
    bep = solve(((x - price_buy) * no_stocks / MARKUP) - total_cost, x)
    price_sell = bep[0]
    print('In a FIXED-cost scenario you can buy {} stocks'.format(no_stocks))
    print('The selling point inc. margin is then {} NOK \n'.format(price_sell))

    fixed_bep = price_sell
    fixed_no_stocks = no_stocks
    fixed_cost_buy = cost_buy
    fixed_cost_sell = cost_sell
    
    ##########################
    # variable cost analysis #
    ##########################
    cost_buy = n * price_buy * VARIABLE_COST
    cost_sell = n * x * VARIABLE_COST
    total_cost = cost_buy + cost_sell

    bep = (((x - price_buy) * n) / MARKUP) - total_cost # no_stocks = n - (balance - total_cost) / price_buy
    bep = solve([bep, n - ((balance - total_cost) / price_buy)], dict = True)

    no_stocks = bep[0][n]
    price_sell = bep[0][x]
    print('In a VARIABLE-cost scenario you can buy {} stocks'.format(no_stocks))
    print('The selling point inc. margin is then {} NOK \n'.format(price_sell))

    var_bep = price_sell
    var_no_stocks = no_stocks
    var_cost_buy = price_buy * no_stocks * VARIABLE_COST
    var_cost_sell = price_sell * no_stocks * VARIABLE_COST
    
    ########################
    # Resolve fixed vs var #
    ########################
    
#     try: 
    if (fixed_cost_buy > var_cost_buy) & (fixed_cost_sell > var_cost_sell):
        print('fixed \n')
        rate_type  = 'fixed'
        total_cost = fixed_cost_buy + fixed_cost_buy
        no_stocks  = math.floor(fixed_no_stocks)
        bep        = solve(((x - price_buy) * no_stocks / MARKUP) - total_cost, x)
        price_sell = bep[0]

    elif (fixed_cost_buy < var_cost_buy) & (fixed_cost_sell < var_cost_sell):
        print('var \n')
        rate_type  = 'variable'
        no_stocks  = math.floor(var_no_stocks)
        bep        = solve(((x - price_buy) * no_stocks / MARKUP) - no_stocks * VARIABLE_COST * (price_buy + x), x)
        price_sell = bep[0]
        total_cost = no_stocks * VARIABLE_COST * (price_buy + price_sell)

    else:
        "We have different variable cost and fixed cost - solve this issue as it arises."
    
#     except:
#         print('Not enough money to purchase a single stock \n')
#         no_stocks = 0
#         price_sell = 0
#         total_cost = 0
    
    print('You pay a {} rate for this transaction.'.format(rate_type))
    print('You can buy: {} stocks, sell them at: {} NOK, at a total cost: {} NOK'
                  .format(no_stocks, round(price_sell,2), round(total_cost, 2)))
    
    return no_stocks, price_sell, total_cost


In [29]:
# init bank account
s = bank_account()  # s.deposit(x), s.withdraw(y), s.display(), s.reset()

# init date-range
start_date = date(2020, 3, 1)
end_date   = date.today()

# init list of intresting stocks
instruments = ['MSFT']

# init other things
no_stocks = 0
s = bank_account()
s.deposit(10000)

df = pd.DataFrame()

for day in daterange(start_date, end_date):
    
    # real code, ran daily
    for instrument in instruments:

        print()
        b, p = buy(instrument, day) # boolean buy, price in dollar
    
        if   b == True:
            print('advice: buy, date: {}, for: {} @ ${}'.format(day, instrument, p)) 
      
            if (s.balance >= 0) & (s.balance > p):

                no_stocks, selling_price, total_cost = bep(s.balance, p)
                
                
                
                cost = total_cost + no_stocks * selling_price
                print('cost totals {}'.format(cost))
                s.balance
                print('the withdrawl is: {} of type {}'.format(cost, type(cost)))
                s.withdraw(cost)
                s.display()
                
    
                print('----------------------------------------------------------------')
                print('On {}, purchase {} stocks from {} @ ${}'.format(day, no_stocks, instrument, p))
                print('Total cost ${}, gives account value after ${}'.format(cost, s.balance))
                print('You will sell off these stocks at ${}'.format(selling_price))
                print('----------------------------------------------------------------')
                
                df = df.append({'order' : 'buy' , 'date' : day, 'price' : p, 
                            'volume' : no_stocks, 'acount_value' : selling_price}, ignore_index=True)

                
            else:
                print('You dont have enough money to buy, namely you have {}'.format(account_value))
                    
        elif b == False: # buy is false
            print("advice: don't buy, date: {}, for: {} @ ${}".format(day, instrument, p))
            
        else:
            print('Error, something went wrong in the buy-loop.')



Amount Deposited: 10000

advice: don't buy, date: 2020-03-01, for: MSFT @ $162.01

advice: don't buy, date: 2020-03-02, for: MSFT @ $162.01

advice: don't buy, date: 2020-03-03, for: MSFT @ $172.79

advice: don't buy, date: 2020-03-04, for: MSFT @ $164.51

advice: don't buy, date: 2020-03-05, for: MSFT @ $170.55

advice: don't buy, date: 2020-03-06, for: MSFT @ $166.27

advice: don't buy, date: 2020-03-07, for: MSFT @ $161.57

advice: don't buy, date: 2020-03-08, for: MSFT @ $161.57

advice: don't buy, date: 2020-03-09, for: MSFT @ $161.57

advice: buy, date: 2020-03-10, for: MSFT @ $150.62
In a FIXED-cost scenario you can buy 66.28734563802948 stocks
The selling point inc. margin is then 150.870274012940 NOK 

In a VARIABLE-cost scenario you can buy 65.0768962943919 stocks
The selling point inc. margin is then 153.816584133401 NOK 

var 

You pay a variable rate for this transaction.
You can buy: 65 stocks, sell them at: 153.82 NOK, at a total cost: 197.88 NOK
cost totals 10195.961748

In [30]:
df

,acount_value,date,order,price,volume
0,153.816584133401,2020-03-10,buy,150.62,65.0
